# Test GPU

In [ ]:
import torch
torch.cuda.current_device()
torch.cuda.get_device_name(0)

'Tesla T4'

# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Copy Files

In [ ]:
import shutil
import os
import glob


shutil.copyfile("/content/drive/My Drive/AlzheimerStallCatcher3DConvPointCloud/resnext.py", "resnext.py")
#shutil.copyfile("/content/drive/My Drive/AlzheimerStallCatcher3DConvPointCloud/densenet.py", "densenet.py")
#shutil.copyfile("/content/drive/My Drive/AlzheimerStallCatcher3DConvPointCloud/wide_resnet.py", "wide_resnet.py")


!jar -xf "/content/drive/My Drive/AlzheimerStallCatcher3DConvPointCloud/traintestlist.zip";

print("Done")

Done


# **Loading Point Cloud Files**

In [ ]:
!jar -xf "/content/drive/My Drive/AlzheimerStallCatcher3DConvPointCloud/Alzheimer Data/micro_1.zip";
print("partition 1 imported")

!jar -xf "/content/drive/My Drive/AlzheimerStallCatcher3DConvPointCloud/Alzheimer Data/micro_2.zip";
print("partition 2 imported")

!jar -xf "/content/drive/My Drive/AlzheimerStallCatcher3DConvPointCloud/Alzheimer Data/micro_3.zip";
print("partition 3 imported")

files = [f for f in glob.glob("micro/" + "*" + ".pt", recursive=True)]
print("Total: " + str(len(files)) + " should be 2399")

partition 1 imported
partition 2 imported
partition 3 imported
Total: 2399 should be 2399


In [ ]:
! rm -rf loss_acc_curve # remove any folder fully

# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc
import time
import os
import glob
import shutil
from sklearn.metrics import matthews_corrcoef as mcc

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.utils.data

torch.manual_seed(100)
np.random.seed(0)

import csv

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools

# Dataset

### Custom Dataset Class

In [ ]:
from torch.utils.data import Dataset, DataLoader

class VoxelDataset(Dataset):
    def __init__(self, dataset_path, split_path, split_number, training):
        self.training = training
        self.sequences, self.labels = self._extract_sequence_paths_and_labels(dataset_path, split_path, split_number,
                                                                              training)  # creating a list of directories where the extracted frames are saved
        self.label_names = ["Non-stalled", "Stalled"]  # Getting the label names or name of the class
        self.num_classes = len(self.label_names)  # Getting the number of class


    def _extract_sequence_paths_and_labels(
            self, dataset_path, split_path="data/traintestlist", split_number=0, training=True
    ):
        """ Extracts paths to sequences given the specified train / test split """
        fn = f"fold_{split_number}_train.csv" if training else f"fold_{split_number}_test.csv"
        split_path = os.path.join(split_path, fn)
        df = pd.read_csv(split_path)
        file_name = df['filename'].values
        all_labels = df['class'].values
        sequence_paths = []
        classes = []
        for i, video_name in enumerate(file_name):
            seq_name = video_name.split(".mp4")[0]
            sequence_paths += [os.path.join(dataset_path, seq_name).replace('\\', '/')]
            classes += [all_labels[i]]
        return sequence_paths, classes

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        sequence_path = self.sequences[index % len(self)]
        target = self.labels[index % len(self)]

        X = torch.load(sequence_path + ".pt")
        #X = scipy.ndimage.zoom(X, (1.0, 1.0, 1.5625, 1.5625), order=1, mode='nearest', prefilter=False)   #to increase spatial size
        #X = torch.from_numpy(X)
        #print(f"type is {type(X)} and dim is {X.shape}")

        return X, target


### Create Dataset

In [ ]:
batch_size = 60

split_number = 3

augmentation_enabled = True   #enabling augmentation or not
augment_volume = 6  #max 15


if augmentation_enabled ==  True:
  batch_size = int(batch_size/augment_volume)


start = time.time()

dataset_path = 'micro'
split_path = 'traintestlist'

depth, height, width = 32, 64, 64

# Define training set

train_dataset_vox = VoxelDataset(
    dataset_path=dataset_path,
    split_path=split_path,
    split_number=split_number,
    training=True,
)

train_dataloader_vox = DataLoader(train_dataset_vox, batch_size= batch_size,shuffle=True, num_workers=4)

# Define test set
test_dataset_vox = VoxelDataset(
    dataset_path=dataset_path,
    split_path=split_path,
    split_number=split_number,
    training=False,
)
test_dataloader_vox = DataLoader(test_dataset_vox, batch_size=batch_size, shuffle=False, num_workers=4)

endtime = time.time()

print("Elapsed time : " + str(endtime-start))

gc.collect() 


Elapsed time : 0.011458396911621094


28

In [ ]:
len(train_dataloader_vox)

180

# Network Model

In [ ]:
#copying weight file to current directory
shutil.copyfile("/content/drive/My Drive/AlzheimerStallCatcher3DConvPointCloud/kinetics_resnext_101_RGB_16_best.pth", "weight_3D.pth")
checkpoint_model = "weight_3D.pth"

shutil.copyfile("/content/drive/My Drive/AlzheimerStallCatcher3DConvPointCloud/3D_ptcl_resnext101_clsballoss_6for_cloud_size_32_128_128_acc_85.524_mcc_0.641.pth", "weight_3D1.pth")
checkpoint_model1 = "weight_3D1.pth"

In [ ]:
'''
##for resnet / resnext with modified attributes

import resnext
#import resnet
class Encoder_pt(nn.Module):
    def __init__(self, num_classes):
        super(Encoder_pt, self).__init__()
      
        rsnxt = resnet.resnet101(
                num_classes=600,
                shortcut_type='B',
                sample_size=64,
                sample_duration=32)
        
        rsnxt = resnext.resnext101(
                num_classes=600,
                shortcut_type='B',
                cardinality=32,
                sample_size=64,
                sample_duration=32)

        ##pretrained weight loading
        rsnxt = rsnxt.cuda()
        rsnxt = nn.DataParallel(rsnxt, device_ids=None)
        pretrain = torch.load(checkpoint_model, map_location=torch.device('cpu'))
        rsnxt.load_state_dict(pretrain['state_dict'])
        

        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.4)
        self.feature_extractor = nn.Sequential(*list(rsnxt.module.children())[0:2])
        self.feature_extractor_new1 = nn.Sequential(*list(rsnxt.module.children())[2:3])
        self.feature_extractor_new2 = nn.Sequential(*list(rsnxt.module.children())[3:4])
        self.feature_extractor_new3 = nn.Sequential(*list(rsnxt.module.children())[4:5])
        self.feature_extractor_new4 = nn.Sequential(*list(rsnxt.module.children())[5:6])
        self.feature_extractor_new5 = nn.Sequential(*list(rsnxt.module.children())[6:7])
        self.feature_extractor_new6 = nn.Sequential(*list(rsnxt.module.children())[7:8])
        self.feature_extractor_new7 = nn.Sequential(*list(rsnxt.module.children())[8:9])
        #self.feature_extractor_new8 = nn.Sequential(*list(resnet.children())[9:10])
        #self.feature_extractor_new8 = nn.Linear(rsnxt.module.fc.in_features, num_classes)
        self.feature_extractor_new8 = nn.Linear(18432, num_classes)
        self.softmax = nn.Softmax()
        

    def forward(self, x):
        #with torch.no_grad():
        x = self.feature_extractor(x)     ##more droputs added
        x = self.dropout1(x)  
        x = self.feature_extractor_new1(x)
        x = self.dropout1(x)
        x = self.feature_extractor_new2(x)
        x = self.dropout1(x)
        x = self.feature_extractor_new3(x)
        x = self.dropout1(x)
        x = self.feature_extractor_new4(x)
        x = self.dropout1(x)
        x = self.feature_extractor_new5(x)
        x = self.dropout1(x)
        x = self.feature_extractor_new6(x)
        x = self.dropout1(x)
        x = self.feature_extractor_new7(x)
        x = x.view(x.shape[0],-1)   #without flattening it can't be passed through dense layer
        x = self.dropout2(x)
        x = self.feature_extractor_new8(x)
        #x = self.softmax(x)
        #x = x.view(x.size(0), -1)
        
        return x
'''

In [ ]:
##for densenet network

import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict


class _DenseLayer(nn.Sequential):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate):
        super().__init__()
        self.add_module('norm1', nn.BatchNorm3d(num_input_features))
        self.add_module('relu1', nn.ReLU(inplace=True))
        self.add_module(
            'conv1',
            nn.Conv3d(num_input_features,
                      bn_size * growth_rate,
                      kernel_size=1,
                      stride=1,
                      bias=False))
        self.add_module('norm2', nn.BatchNorm3d(bn_size * growth_rate))
        self.add_module('relu2', nn.ReLU(inplace=True))
        self.add_module(
            'conv2',
            nn.Conv3d(bn_size * growth_rate,
                      growth_rate,
                      kernel_size=3,
                      stride=1,
                      padding=1,
                      bias=False))
        self.drop_rate = drop_rate

    def forward(self, x):
        new_features = super().forward(x)
        if self.drop_rate > 0:
            new_features = F.dropout(new_features,
                                     p=self.drop_rate,
                                     training=self.training)
        return torch.cat([x, new_features], 1)


class _DenseBlock(nn.Sequential):

    def __init__(self, num_layers, num_input_features, bn_size, growth_rate,
                 drop_rate):
        super().__init__()
        for i in range(num_layers):
            layer = _DenseLayer(num_input_features + i * growth_rate,
                                growth_rate, bn_size, drop_rate)
            self.add_module('denselayer{}'.format(i + 1), layer)


class _Transition(nn.Sequential):

    def __init__(self, num_input_features, num_output_features):
        super().__init__()
        self.add_module('norm', nn.BatchNorm3d(num_input_features))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module(
            'conv',
            nn.Conv3d(num_input_features,
                      num_output_features,
                      kernel_size=1,
                      stride=1,
                      bias=False))
        self.add_module('pool', nn.AvgPool3d(kernel_size=2, stride=2))


class DenseNet(nn.Module):
    """Densenet-BC model class
    Args:
        growth_rate (int) - how many filters to add each layer (k in paper)
        block_config (list of 4 ints) - how many layers in each pooling block
        num_init_features (int) - the number of filters to learn in the first convolution layer
        bn_size (int) - multiplicative factor for number of bottle neck layers
          (i.e. bn_size * k features in the bottleneck layer)
        drop_rate (float) - dropout rate after each dense layer
        num_classes (int) - number of classification classes
    """

    def __init__(self,
                 n_input_channels=3,
                 conv1_t_size=7,
                 conv1_t_stride=1,
                 no_max_pool=False,
                 growth_rate=32,
                 block_config=(6, 12, 24, 16),
                 num_init_features=64,
                 bn_size=4,
                 drop_rate=0,
                 num_classes=1000):

        super().__init__()

        # First convolution
        self.features = [('conv1',
                          nn.Conv3d(n_input_channels,
                                    num_init_features,
                                    kernel_size=(conv1_t_size, 7, 7),
                                    stride=(conv1_t_stride, 2, 2),
                                    padding=(conv1_t_size // 2, 3, 3),
                                    bias=False)),
                         ('norm1', nn.BatchNorm3d(num_init_features)),
                         ('relu1', nn.ReLU(inplace=True))]
        if not no_max_pool:
            self.features.append(
                ('pool1', nn.MaxPool3d(kernel_size=3, stride=2, padding=1)))
        self.features = nn.Sequential(OrderedDict(self.features))

        # Each denseblock
        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(num_layers=num_layers,
                                num_input_features=num_features,
                                bn_size=bn_size,
                                growth_rate=growth_rate,
                                drop_rate=drop_rate)
            self.features.add_module('denseblock{}'.format(i + 1), block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = _Transition(num_input_features=num_features,
                                    num_output_features=num_features // 2)
                self.features.add_module('transition{}'.format(i + 1), trans)
                num_features = num_features // 2

        # Final batch norm
        self.features.add_module('norm5', nn.BatchNorm3d(num_features))

        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')
            elif isinstance(m, nn.BatchNorm3d) or isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

        # Linear layer
        self.classifier = nn.Linear(num_features, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                nn.init.kaiming_normal_(m.weight,
                                        mode='fan_out',
                                        nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm3d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.adaptive_avg_pool3d(out,
                                    output_size=(1, 1,
                                                 1)).view(features.size(0), -1)
        out = self.classifier(out)
        return out


def generate_model(model_depth, **kwargs):
    assert model_depth in [121, 169, 201, 264]

    if model_depth == 121:
        model = DenseNet(num_init_features=64,
                         growth_rate=32,
                         block_config=(6, 12, 24, 16),
                         **kwargs)
    elif model_depth == 169:
        model = DenseNet(num_init_features=64,
                         growth_rate=32,
                         block_config=(6, 12, 32, 32),
                         **kwargs)
    elif model_depth == 201:
        model = DenseNet(num_init_features=64,
                         growth_rate=32,
                         block_config=(6, 12, 48, 32),
                         **kwargs)
    elif model_depth == 264:
        model = DenseNet(num_init_features=64,
                         growth_rate=32,
                         block_config=(6, 12, 64, 48),
                         **kwargs)

    return model


In [ ]:
#for densenet 3D
model = generate_model(model_depth = 121 , num_classes = 2)
model

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:130: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


DenseNet(
  (features): Sequential(
    (conv1): Conv3d(3, 64, kernel_size=(7, 7, 7), stride=(1, 2, 2), padding=(3, 3, 3), bias=False)
    (norm1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (pool1): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv3d(64, 128, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (norm2): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv3d(128, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm3d(96, eps=1e-05, momentum=0.1, affine=True, track_running_s

In [ ]:
#for resnext (ground up training)
import resnext
model = resnext.resnext152(
                num_classes=2,
                shortcut_type='B',
                cardinality=32,
                sample_size=64,
                sample_duration=32)


/content/resnext.py:129: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


In [ ]:
##for wide_resnet50(ground up training)
import wide_resnet

model = wide_resnet.wide_resnet50(num_classes=2 , k=1,
                                         sample_size=64, sample_duration=32,
                                         last_fc=True)

In [ ]:
#from apex import amp

use_amp = False    #Using automatic mixed precision

# model = Encoder_pt(
#     num_classes=2,
# )

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


learning_rate = 4e-4
wd = 5e-4

'''
### Optimizer ####
if RESUME_TRAINING:
  learning_rate = 2e-4
  wd = 6e-4
else:
'''

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay= wd)  #added L2 regularization

##for loading pre-trained weight/attributes
checkpoint = torch.load(checkpoint_model1)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])


'''
##if amp is used for mixed precision

opt_level = "O2"

##For first stage
if use_amp:
  model, optimizer = amp.initialize(model, optimizer, opt_level=opt_level)


##For 2nd stage
if RESUME_TRAINING:
  if use_amp:
    checkpoint = torch.load(checkpoint_model1)  ##checkpoint_model1 is the pretrained load in first stage
    model, optimizer = amp.initialize(model, optimizer, opt_level=opt_level)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    amp.load_state_dict(checkpoint['amp'])
  
  else:
    #File name of previously trained weight file
    #checkpoint_model1="model_checkpoints/3D_pointcloud_resnet101_0for_cloud_size_32_64_64_acc_32.779_mcc_-0.014.pth"
    checkpoint = torch.load(checkpoint_model1)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    #epoch = checkpoint['epoch']
    #loss = checkpoint['loss']

'''
#device
model

ResNeXt(
  (conv1): Conv3d(3, 64, kernel_size=(7, 7, 7), stride=(1, 2, 2), padding=(3, 3, 3), bias=False)
  (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResNeXtBottleneck(
      (conv1): Conv3d(64, 128, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), groups=32, bias=False)
      (bn2): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv3d(128, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn3): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequentia

### Model Hyperparameters

In [ ]:
from torch.optim import *
#error = nn.CrossEntropyLoss().to(device)

num_epochs = 60
step_size = 15
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size, gamma=0.7, last_epoch=-1)

**Class Balance loss**

In [ ]:
import torch.nn.functional as F
def CB_loss(labels, logits, samples_per_cls, no_of_classes, loss_type, beta, gamma):
    """Compute the Class Balanced Loss between `logits` and the ground truth `labels`.
    Class Balanced Loss: ((1-beta)/(1-beta^n))*Loss(labels, logits)
    where Loss is one of the standard losses used for Neural Networks.
    Args:
      labels: A int tensor of size [batch].
      logits: A float tensor of size [batch, no_of_classes].
      samples_per_cls: A python list of size [no_of_classes].
      no_of_classes: total number of classes. int
      loss_type: string. One of "sigmoid", "focal", "softmax".
      beta: float. Hyperparameter for Class balanced loss.
      gamma: float. Hyperparameter for Focal loss.
    Returns:
      cb_loss: A float tensor representing class balanced loss
    """
    effective_num = 1.0 - np.power(beta, samples_per_cls)
    weights = (1.0 - beta) / np.array(effective_num)
    weights = weights / np.sum(weights) * no_of_classes

    labels_one_hot = F.one_hot(labels, no_of_classes).float()

    weights = torch.tensor(weights).float().to(device)
    weights = weights.unsqueeze(0)
    weights = weights.repeat(labels_one_hot.shape[0],1) * labels_one_hot
    weights = weights.sum(1)
    weights = weights.unsqueeze(1)
    weights = weights.repeat(1,no_of_classes)

    if loss_type == "focal":
        cb_loss = focal_loss(labels_one_hot, logits, weights, gamma)
    elif loss_type == "sigmoid":
        cb_loss = F.binary_cross_entropy_with_logits(input = logits,target = labels_one_hot, weights = weights)
    elif loss_type == "softmax":
        pred = logits.softmax(dim = 1)
        cb_loss = F.binary_cross_entropy(input = pred, target = labels_one_hot, weight = weights)
    return cb_loss



no_of_classes = 2
beta = 0.9999
gamma = 2.0
samples_per_cls = [42,18]   #in batch size of 60 we have class ration of 70:30
loss_type = "softmax"


# Visualization and Saving Output

In [ ]:
def conf_mat(cf, epoch, acc, mcc, y_true, train_loss, train_acc, test_loss, test_acc):
  try:
    plt.imshow(cf,cmap=plt.cm.RdYlGn,interpolation='nearest')
    plt.colorbar()
    plt.title('Confusion Matrix without Normalization')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    tick_marks = np.arange(len(set(y_true))) # length of classes
    class_labels = ['Non Stalled','Stalled']
    tick_marks
    plt.xticks(tick_marks,class_labels)
    plt.yticks(tick_marks,class_labels)
    # plotting text value inside cells
    thresh = cf.max() / 2.
    for i,j in itertools.product(range(cf.shape[0]),range(cf.shape[1])):
        plt.text(j,i,format(cf[i,j],'d'),horizontalalignment='center',color='white' if cf[i,j] >thresh else 'black')
    #plt.show()
    os.makedirs("confusion_matrix", exist_ok=True)
    plt.savefig(f"confusion_matrix/epoch{epoch}_accuracy{round(acc.item(),3)}_mcc_{round(mcc.item(),3)}.png")
    plt.close('all')

    os.makedirs("loss_acc_curve", exist_ok=True)

    ##creating subplot for loss,acc
    fig1, axs = plt.subplots(4,sharex=True,constrained_layout=True)
    axs[0].plot(train_loss, color = "red") 
    axs[0].set_title("Train loss")
    axs[1].plot(train_acc, color = "blue") 
    axs[1].set_title("Train Accuracy")
    axs[2].plot(test_loss, color = "green")
    axs[2].set_title("Test Loss")
    axs[3].plot(test_acc) 
    axs[3].set_title("Test Accuracy")
    #fig1.tight_layout()
    #plt.show()
    fig1.savefig(f"loss_acc_curve/epoch{epoch}_accuracy{round(acc.item(),3)}_mcc_{round(mcc.item(),3)}.png")
    plt.close(fig1)
  except:
    print("MCC is 0")

In [ ]:
#for cross entropy loss
def validation(epoch,train_loss, train_acc, test_loss, test_acc):

  correct = 0
  total = 0
  t_loss = 0
  y_true = []
  y_pred = []

  model.eval()

  for images, labels in test_dataloader_vox:

      y_true = np.append(y_true, labels.numpy())

      images = images.view(-1,3,depth,height,width)
      
      test = Variable(images.to(device), requires_grad=False)
      labels = Variable(labels.to(device), requires_grad=False)

      #test = test.half()

      with torch.no_grad():
      
        # Forward propagation
        outputs = model(test)
        #t_loss += error(outputs, labels)
        t_loss += CB_loss(labels, outputs, samples_per_cls, no_of_classes,loss_type, beta, gamma)

        # Get predictions from the maximum value
        predicted = torch.max(outputs.data, 1)[1]
        #print(f"prediction size are {predicted.shape}")
        y_pred = np.append(y_pred, predicted.cpu().numpy())
        # Total number of labels
        total += len(labels)
        correct += (predicted == labels).sum()
  model.train()
  loss = t_loss.cpu().numpy() / float(total)
  test_loss.append(loss)
  accuracy = 100 * correct / float(total)
  test_acc.append(accuracy)

  mcc_score = mcc(y_true, y_pred)
  print(f"MCC score is {round(mcc_score,4)}")


  global MCC_SCORE
  if MCC_SCORE < mcc_score:
      MCC_SCORE = mcc_score
      os.makedirs("model_checkpoints", exist_ok=True)
      try:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, f"model_checkpoints/3D_ptcl_rsnxt152_clsballoss_{epoch}for_cloud_size_32_64_64_acc_{round(accuracy.item(),3)}_mcc_{round(mcc_score.item(),3)}.pth")
      except:
        pass
  cf =confusion_matrix(y_true, y_pred)
  #print(cf)
  conf_mat(cf, epoch, accuracy, mcc_score, y_true, train_loss, train_acc, test_loss, test_acc)

  print('TESTING ---> Epoch: {} Loss: {} Accuracy: {} %'.format(epoch, round(loss,3), round(accuracy.item(),3)))


  return test_loss, test_acc


# Training

**TIP:** This training could take several hours depending on how many iterations you chose in the .cfg file. You will want to let this run as you sleep or go to work for the day, etc. However, Colab Cloud Service kicks you off it's VMs if you are idle for too long (30-90 mins).

To avoid this hold (CTRL + SHIFT + i) at the same time to open up the inspector view on your browser.

Paste the following code into your console window and hit **Enter**
```
function ClickConnect(){
console.log("Working"); 
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click() 
}
setInterval(ClickConnect,60000)
```

### Augmentation Code

In [ ]:
def augment(images, labels, augment_volume=2):

  [instances, channel, depth, height, width] = images.shape

  images_aug = torch.zeros((instances*augment_volume, channel, depth, height, width))
  labels_aug = torch.zeros(instances*augment_volume)

  for aug_type in range(augment_volume):
    augmented = im_aug(images, aug_type)
    
    images_aug[aug_type*instances:(aug_type+1)*instances, :, :, :, :] = augmented
    labels_aug[aug_type*instances:(aug_type+1)*instances] = labels

  return images_aug.float(), labels_aug.long()



def im_aug(images, aug_type):

    if aug_type == 0:
      return images

    elif aug_type == 1:
      return torch.rot90(images, 1, [3,4])

    elif aug_type == 2:
      return torch.rot90(images, 2, [3,4])

    elif aug_type == 3:
      return torch.rot90(images, 3, [3,4])

    elif aug_type == 4:
      temp = torch.flip(images, [2,3])
      return temp

    elif aug_type == 5:
      temp = torch.flip(images, [2,3])
      return torch.rot90(temp, 1, [3,4])

    elif aug_type == 6:
      temp = torch.flip(images, [2,3])
      return torch.rot90(temp, 2, [3,4])

    elif aug_type == 7:
      temp = torch.flip(images, [2,3])
      return torch.rot90(temp, 3, [3,4])

    elif aug_type == 8:
      temp = torch.transpose(images, 3, 4)
      return temp

    elif aug_type == 9:
      temp = torch.transpose(images, 3, 4)
      return torch.rot90(temp, 1, [3,4])

    elif aug_type == 10:
      temp = torch.transpose(images, 3, 4)
      return torch.rot90(temp, 2, [3,4])

    elif aug_type == 11:
      temp = torch.transpose(images, 3, 4)
      return torch.rot90(temp, 3, [3,4])

    elif aug_type == 12:
      temp = torch.flip(images, [2,3])
      temp = torch.transpose(temp, 3, 4)
      return temp

    elif aug_type == 13:
      temp = torch.flip(images, [2,3])
      temp = torch.transpose(temp, 3, 4)
      return torch.rot90(temp, 1, [3,4])

    elif aug_type == 14:
      temp = torch.flip(images, [2,3])
      temp = torch.transpose(temp, 3, 4)
      return torch.rot90(temp, 2, [3,4])

    elif aug_type == 15:
      temp = torch.flip(images, [2,3])
      temp = torch.transpose(temp, 3, 4)
      return torch.rot90(temp, 3, [3,4])


# Train

In [ ]:
### TRAINING code
#num_epochs = 50
#import scipy
MCC_SCORE = 0


train_loss = []   #to keep track of loss with respect to number of epoch 
test_loss = []
iteration_list = []
train_acc = []
test_acc = []

for epoch in tqdm(range(num_epochs)):
    #count = 1
    accuracy_list = []
    loss_list = []
    for i, (images, labels) in tqdm(enumerate(train_dataloader_vox)):
        correct = 0
        
        if augmentation_enabled == True:
          images, labels = augment(images, labels, augment_volume)

        images = images.view(-1,3,depth,height,width)
        
        train_img = Variable(images.to(device), requires_grad=True)
        labels = Variable(labels.to(device), requires_grad=False)

        #train_img = train_img.half()
        # Clear gradients
        optimizer.zero_grad()
        # Forward propagation
        outputs = model(train_img)
        # Calculate softmax and ross entropy loss
        #loss = error(outputs, labels)   #for cross-entropy loss
        loss = CB_loss(labels, outputs, samples_per_cls, no_of_classes,loss_type, beta, gamma)

        
        #labels = labels.view(-1, 1)  #only for BCELogitsloss
        #loss = error(outputs.float(), labels.float())  #if BCELoss is measured
        
        # Calculating gradients
        loss.backward()
        # Update parameters
        optimizer.step()
        
        predicted = torch.max(outputs.data, 1)[1]   
        correct = (predicted == labels).sum()
        total = len(labels)        
        accuracy = 100 * correct / float(total)

        '''
        ###For BCELoss
        predicted = torch.tensor([0 if i<=0.5 else 1 for i in outputs]).to(device)
        accuracy = 100 * (predicted.detach() == labels).cpu().numpy().mean()
        '''

        accuracy_list = np.append(accuracy_list, accuracy.cpu().numpy())
        loss_list = np.append(loss_list, loss.detach().cpu().numpy())
        #print(f"Training Mini Batch--> Epoch:{epoch} Iteration:{count} Loss :{loss} Accuracy: {accuracy}")
        #count += 1

    final_loss = np.mean(loss_list)
    final_acc = np.mean(accuracy_list)
    print(f"TRAINING ---> Epoch: {epoch} Loss: {round(final_loss,4)} Accuracy: {round(final_acc,4)}")
    train_loss.append(final_loss)
    train_acc.append(final_acc)
    gc.collect() 
    test_loss , test_acc = validation(epoch, train_loss, train_acc, test_loss, test_acc)
    
    scheduler.step()
    



  0%|          | 0/60 [00:00<?, ?it/s]


0it [00:00, ?it/s]


1it [00:03,  3.41s/it]


2it [00:06,  3.35s/it]


3it [00:09,  3.31s/it]


4it [00:13,  3.28s/it]


5it [00:16,  3.26s/it]


6it [00:19,  3.25s/it]


7it [00:22,  3.25s/it]


8it [00:25,  3.24s/it]


9it [00:29,  3.24s/it]


10it [00:32,  3.23s/it]


11it [00:35,  3.22s/it]


12it [00:38,  3.22s/it]


13it [00:42,  3.22s/it]


14it [00:45,  3.21s/it]


15it [00:48,  3.21s/it]


16it [00:51,  3.21s/it]


17it [00:54,  3.23s/it]


18it [00:58,  3.22s/it]


19it [01:01,  3.21s/it]


20it [01:04,  3.22s/it]


21it [01:07,  3.23s/it]


22it [01:11,  3.24s/it]


23it [01:14,  3.25s/it]


24it [01:17,  3.24s/it]


25it [01:20,  3.23s/it]


26it [01:23,  3.22s/it]


27it [01:27,  3.22s/it]


28it [01:30,  3.23s/it]


29it [01:33,  3.23s/it]


30it [01:36,  3.22s/it]


31it [01:40,  3.22s/it]


32it [01:43,  3.22s/it]


33it [01:46,  3.22s/it]


34it [01:49,  3.23s/it]


35it [01:52,  3.23s/it]


36it [01:56,  3.22s/it]


37it [01:

TRAINING ---> Epoch: 0 Loss: 0.7031 Accuracy: 52.4444


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


  2%|▏         | 1/60 [09:49<9:39:43, 589.56s/it]

MCC score is 0.0
MCC is 0
TESTING ---> Epoch: 0 Loss: 0.058 Accuracy: 70.618 %





0it [00:00, ?it/s]


1it [00:03,  3.43s/it]


2it [00:06,  3.37s/it]


3it [00:09,  3.32s/it]


4it [00:13,  3.28s/it]


5it [00:16,  3.26s/it]


6it [00:19,  3.26s/it]


7it [00:22,  3.26s/it]


8it [00:25,  3.25s/it]


9it [00:29,  3.25s/it]


10it [00:32,  3.24s/it]


11it [00:35,  3.24s/it]


12it [00:38,  3.25s/it]


13it [00:42,  3.25s/it]


14it [00:45,  3.25s/it]


15it [00:48,  3.25s/it]


16it [00:51,  3.25s/it]


17it [00:55,  3.24s/it]


18it [00:58,  3.23s/it]


19it [01:01,  3.23s/it]


20it [01:04,  3.23s/it]


21it [01:08,  3.23s/it]


22it [01:11,  3.23s/it]


23it [01:14,  3.23s/it]


24it [01:17,  3.23s/it]


25it [01:20,  3.22s/it]


26it [01:24,  3.23s/it]


27it [01:27,  3.23s/it]


28it [01:30,  3.23s/it]


29it [01:33,  3.23s/it]


30it [01:37,  3.24s/it]


31it [01:40,  3.24s/it]


32it [01:43,  3.23s/it]


33it [01:46,  3.24s/it]


34it [01:50,  3.24s/it]


35it [01:53,  3.24s/it]


36it [01:56,  3.23s/it]


37it [01:59,  3.24s/it]


38it [02:03,  3.24s/it]

...

TRAINING ---> Epoch: 23 Loss: 0.1779 Accuracy: 91.3148
MCC score is 0.5937




 40%|████      | 24/60 [3:55:05<5:53:48, 589.68s/it]

TESTING ---> Epoch: 23 Loss: 0.043 Accuracy: 81.135 %





0it [00:00, ?it/s]


1it [00:03,  3.47s/it]


2it [00:06,  3.40s/it]


3it [00:09,  3.35s/it]


4it [00:13,  3.32s/it]


5it [00:16,  3.30s/it]


6it [00:19,  3.28s/it]


7it [00:22,  3.27s/it]


8it [00:26,  3.25s/it]


9it [00:29,  3.24s/it]


10it [00:32,  3.24s/it]


11it [00:35,  3.24s/it]


12it [00:39,  3.23s/it]


13it [00:42,  3.23s/it]


14it [00:45,  3.24s/it]


15it [00:48,  3.24s/it]


16it [00:52,  3.23s/it]


17it [00:55,  3.24s/it]


18it [00:58,  3.24s/it]


19it [01:01,  3.25s/it]


20it [01:05,  3.25s/it]


21it [01:08,  3.25s/it]


22it [01:11,  3.23s/it]


23it [01:14,  3.23s/it]


24it [01:17,  3.23s/it]


25it [01:21,  3.23s/it]


26it [01:24,  3.23s/it]


27it [01:27,  3.23s/it]


28it [01:30,  3.23s/it]


29it [01:34,  3.23s/it]


30it [01:37,  3.23s/it]


31it [01:40,  3.23s/it]


32it [01:43,  3.23s/it]


33it [01:47,  3.25s/it]


34it [01:50,  3.25s/it]


35it [01:53,  3.25s/it]


36it [01:56,  3.24s/it]


37it [01:59,  3.24s/it]


38it [02:03,  3.24s/it]

KeyboardInterrupt: ignored